# Benchmarks

## Prep 

In [1]:
import numpy as np
from sklearn.metrics import accuracy_score

from cnn.utils.graph_manipulation import load_frozen_graph, predict_from_frozen
from cnn.utils.dataset import load_cifar10, dataset_preprocessing_by_keras

/home/m/myvenv3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
pyperf = !find ~ -iname pyperf 2>/dev/null
pyperf = pyperf[0]

In [3]:
nets = ["dense", "conv", "medium"]
names = [[n + "_opt", n + "_quant"] for n in nets]
frozen_nets = [["models/" + n + ".pb"  for n in both] for both in names]
results_files = [["results/" + n + ".json"  for n in both] for both in names]

## Check accuracy 

In [6]:
_, _, x_test, t_test = load_cifar10()
x_test = dataset_preprocessing_by_keras(x_test)

In [7]:
for pb_files in frozen_nets:
    for m in pb_files:
        graph = load_frozen_graph(m)
        out = predict_from_frozen(graph, [x_test], ["features"], ["classes:0", "softmax:0"])
        classes = np.concatenate([batch[0] for batch in out])
        acc = accuracy_score(np.argmax(t_test, axis=1), classes)
        print(f"{m:22}: {acc:6} accuracy")

models/dense_opt.pb   : 0.6358 accuracy
models/dense_quant.pb :  0.622 accuracy
models/conv_opt.pb    : 0.7345 accuracy
models/conv_quant.pb  : 0.7348 accuracy
models/medium_opt.pb  : 0.6461 accuracy
models/medium_quant.pb: 0.6476 accuracy


## Time comparisons 

In [4]:
for r in results_files:
    !{pyperf} show {r[0]} --quiet

1-batch: Mean +- std dev: 2.78 sec +- 0.02 sec
16-batch: Mean +- std dev: 875 ms +- 7 ms
64-batch: Mean +- std dev: 675 ms +- 2 ms
1-batch: Mean +- std dev: 1.44 sec +- 0.04 sec
16-batch: Mean +- std dev: 742 ms +- 34 ms
64-batch: Mean +- std dev: 738 ms +- 11 ms
1-batch: Mean +- std dev: 2.95 sec +- 0.04 sec
16-batch: Mean +- std dev: 1.40 sec +- 0.07 sec
64-batch: Mean +- std dev: 1.25 sec +- 0.00 sec


In [5]:
for r in results_files:
    !{pyperf} compare_to {r[0]} {r[1]} --table

+-----------+-----------+--------------------------------+
| Benchmark | dense_opt | dense_quant                    |
+===========+===========+================================+
| 1-batch   | 2.78 sec  | 7.92 sec: 2.85x slower (+185%) |
+-----------+-----------+--------------------------------+
| 16-batch  | 875 ms    | 1.78 sec: 2.03x slower (+103%) |
+-----------+-----------+--------------------------------+
| 64-batch  | 675 ms    | 1.40 sec: 2.08x slower (+108%) |
+-----------+-----------+--------------------------------+
+-----------+----------+--------------------------------+
| Benchmark | conv_opt | conv_quant                     |
+===========+==========+================================+
| 1-batch   | 1.44 sec | 4.74 sec: 3.30x slower (+230%) |
+-----------+----------+--------------------------------+
| 16-batch  | 742 ms   | 2.78 sec: 3.75x slower (+275%) |
+-----------+----------+--------------------------------+
| 64-batch  | 738 ms   | 2.58 sec: 3.50x slower (+250%) |
+----